In [1]:
import os, random, math, time
from pathlib import Path
from tqdm import tqdm
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms, models
from torchvision.transforms import RandAugment
import timm 
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from collections import Counter
from torch.autograd import Function

# --- 1. SETUP AND HYPERPARAMETERS (From Notebook Cell 4 & 5) ---
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if device == "cuda": torch.cuda.manual_seed_all(SEED)

IMG_SIZE = 224
BATCH_SIZE = 8         
EPOCHS = 15
NUM_WORKERS = 4         
LR = 3e-4              
LABEL_SMOOTH = 0.1
SAVE_PATH = "best_model.pth"
USE_SEGMENTATION = True 

# Loss weights 
ALPHA_DOM = 0.5
BETA_SUPCON = 0.3
ETA_CONS = 0.1
GAMMA_SEG = 0.5

# Mixup/CutMix probabilities and alphas
PROB_MIXUP = 0.5
PROB_CUTMIX = 0.5
MIXUP_ALPHA = 0.2
CUTMIX_ALPHA = 1.0

# Warmup epochs and accumulation steps
WARMUP_EPOCHS = 5
EARLY_STOPPING_PATIENCE = 7
FREEZE_EPOCHS = 5
ACCUMULATION_STEPS = 4

# --- 2. TRANSFORMS (From Notebook Cell 6) ---
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=0.05):
        self.mean = mean
        self.std = std
    def __call__(self, tensor):
        noise = torch.randn(tensor.size()) * self.std + self.mean
        noisy_tensor = tensor + noise
        return torch.clamp(noisy_tensor, 0., 1.)
    def __repr__(self):
        return self.__class__.__name__ + f'(mean={self.mean}, std={self.std})'

weak_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    AddGaussianNoise(0., 0.02),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
strong_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ColorJitter(0.4,0.4,0.4,0.1),
    RandAugment(num_ops=2, magnitude=9),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    AddGaussianNoise(0., 0.05),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
val_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# --- 3. DATASET HELPER FUNCTIONS (From Notebook Cell 7) ---
def read_file_with_encoding(file_path, encodings=['utf-8', 'utf-8-sig', 'ISO-8859-1']):
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as f:
                return f.readlines()
        except UnicodeDecodeError:
            pass
        except Exception as e:
            raise 
    raise RuntimeError(f"Unable to read {file_path} with any of the provided encodings.")

def load_testing_dataset_info(info_file, image_dir):
    image_paths = []
    labels = []
    encodings = ['utf-8-sig', 'utf-8', 'ISO-8859-1', 'latin-1']
    lines = []
    
    for encoding in encodings:
        try:
            with open(info_file, 'r', encoding=encoding) as f:
                lines = f.readlines()
            break
        except UnicodeDecodeError:
            pass
        except Exception as e:
            raise 

    for line in lines:
        parts = line.strip().split()
        if len(parts) == 2:
            image_filename = parts[0]
            try:
                label = int(parts[1])
            except ValueError:
                continue
                
            label = 1 if label == 1 else 0
            image_full_path = os.path.join(image_dir, image_filename)
            image_paths.append(image_full_path)
            labels.append(label)
    
    return image_paths, labels

# --- 4. MULTIDATASET CLASS (From Notebook Cell 7) ---
class MultiDataset(Dataset):
    def __init__(self, root_dirs, txt_files, testing_image_paths=None, testing_labels=None, weak_transform=None, strong_transform=None, use_masks=True):
        self.root_dirs = root_dirs
        self.weak_transform = weak_transform
        self.strong_transform = strong_transform
        self.use_masks = use_masks
        self.samples = []

        if testing_image_paths is not None and testing_labels is not None:
            for img_path, label in zip(testing_image_paths, testing_labels):
                self.samples.append((img_path, label)) 
        
        if isinstance(txt_files, str):
            txt_files = [txt_files]

        all_lines = []
        for t in txt_files:
            if not os.path.exists(t):
                raise RuntimeError(f"TXT file not found: {t}. Please ensure all Kaggle input datasets are mounted.")

            lines = read_file_with_encoding(t)
            all_lines.extend([(line.strip(), t) for line in lines if line.strip()])

        for line, src_txt in all_lines:
            parts = line.split()
            if len(parts) == 0:
                continue

            fname = parts[0]
            if len(parts) >= 2:
                try:
                    lbl = int(parts[1])
                except:
                    lbl = 1 if "CAM" in fname or "cam" in fname else 0
            else:
                lbl = 1 if "CAM" in fname or "cam" in fname else 0
            
            lbl = 1 if lbl == 1 else 0
            base_fname = os.path.basename(fname)  

            found = False
            search_subs = [
                "", "Image", "Imgs", "images", "JPEGImages", "img", 
                "Images/Train", "Images/Test",
            ]
            
            for rdir in self.root_dirs:
                for sub in search_subs:
                    img_path = os.path.join(rdir, sub, base_fname)
                    if os.path.exists(img_path):
                        self.samples.append((img_path, lbl, rdir))
                        found = True
                        break
                if found:
                    break

        if len(self.samples) == 0:
            raise RuntimeError(f"No valid samples found from {txt_files}")

        print(f"✅ Loaded {len(self.samples)} samples from {len(self.root_dirs)} root directories.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        global IMG_SIZE 
        
        sample = self.samples[idx]
        img_path = sample[0]
        lbl = sample[1]
        
        if len(sample) == 3:
            rdir = sample[2]
        else:
            rdir = os.path.dirname(os.path.dirname(img_path))
           
        try:
            img = Image.open(img_path).convert("RGB")
        except:
            img = Image.fromarray(np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8))
            
        if self.weak_transform:
            weak = self.weak_transform(img)
        else:
            weak = transforms.ToTensor()(img)
            
        if self.strong_transform:
            strong = self.strong_transform(img)
        else:
            strong = weak.clone()

        mask = None
        if self.use_masks:
            mask_name = os.path.splitext(os.path.basename(img_path))[0] + ".png"
            found_mask = False
            for mask_dir in ["GT_Object", "GT", "masks", "Mask"]:
                mask_path = os.path.join(rdir, mask_dir, mask_name)
                
                if os.path.exists(mask_path):
                    m = Image.open(mask_path).convert("L").resize((IMG_SIZE, IMG_SIZE))
                    m = np.array(m).astype(np.float32) / 255.0
                    mask = torch.from_numpy((m > 0.5).astype(np.float32)).unsqueeze(0)
                    found_mask = True
                    break

            if mask is None:
                mask = torch.zeros((1, IMG_SIZE, IMG_SIZE), dtype=torch.float32)
                
        return weak, strong, lbl, mask

# --- 5. BACKBONE EXTRACTORS (From Notebook Cell 9) ---

# --- UPDATED CONSTANTS ---
# DenseNet169 final features: 1664
# MobileNetV3 Large final features before global pool: 112 (or 160/960 depending on layer, 
# but based on your previous code 112 is the expected mid-layer output)
# --- UPDATED CONSTANTS FOR TRIPLE FUSION ---
# Final feature maps after Global Average Pooling:
DENSE_CHANNELS = 1920      # DenseNet201
INCEPTION_CHANNELS = 2048  # InceptionV3
EFFICIENT_CHANNELS = 1280  # EfficientNetV2-S (timm default)

TOTAL_FEATURES = DENSE_CHANNELS + INCEPTION_CHANNELS + EFFICIENT_CHANNELS 

class DenseNet201Extractor(nn.Module):
    def __init__(self, pretrained=True):
        super().__init__()
        self.features = models.densenet201(weights='IMAGENET1K_V1' if pretrained else None).features
    def forward(self, x):
        return self.features(x)

class InceptionExtractor(nn.Module):
    def __init__(self, pretrained=True):
        super().__init__()
        # aux_logits=False is necessary to simplify the output to just features
        self.model = models.inception_v3(weights='IMAGENET1K_V1' if pretrained else None, aux_logits=True)
    def forward(self, x):
        # InceptionV3 requires 299x299 ideally, but will work with 224x224
        # We extract features before the final FC layer
        x = self.model.Conv2d_1a_3x3(x)
        x = self.model.Conv2d_2a_3x3(x)
        x = self.model.Conv2d_2b_3x3(x)
        x = self.model.maxpool1(x)
        x = self.model.Conv2d_3b_1x1(x)
        x = self.model.Conv2d_4a_3x3(x)
        x = self.model.maxpool2(x)
        x = self.model.Mixed_5b(x)
        x = self.model.Mixed_5c(x)
        x = self.model.Mixed_5d(x)
        x = self.model.Mixed_6a(x)
        x = self.model.Mixed_6b(x)
        x = self.model.Mixed_6c(x)
        x = self.model.Mixed_6d(x)
        x = self.model.Mixed_6e(x)
        x = self.model.Mixed_7a(x)
        x = self.model.Mixed_7b(x)
        x = self.model.Mixed_7c(x)
        return x

class EfficientNetExtractor(nn.Module):
    def __init__(self, pretrained=True):
        super().__init__()
        # Using timm for a modern EfficientNetV2 implementation
        self.model = timm.create_model('tf_efficientnetv2_s', pretrained=pretrained, num_classes=0, global_pool='')
    def forward(self, x):
        return self.model(x)



# --- 6. USER'S Keras-Style Fusion Model (Corrected) ---


NUM_CLASSES = 2 

class TripleFusionModel(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        
        self.densenet_base = DenseNet201Extractor(pretrained=True)
        self.inception_base = InceptionExtractor(pretrained=True)
        self.efficient_base = EfficientNetExtractor(pretrained=True)
        
        # Freezing logic for the first phase
        for base in [self.densenet_base, self.inception_base, self.efficient_base]:
            for param in base.parameters():
                param.requires_grad = False
            
        self.global_pool = nn.AdaptiveAvgPool2d(1)

        # Classifier input: 1920 + 2048 + 1280 = 5248
        self.classifier = nn.Sequential(
            nn.Linear(TOTAL_FEATURES, 1024), 
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes) 
        )
        
    def forward(self, x):
        # 1. Feature Extraction
        feat_dense = self.densenet_base(x)
        feat_inc = self.inception_base(x)
        feat_eff = self.efficient_base(x)

        # 2. Global Average Pooling & Flattening
        x1 = self.global_pool(feat_dense).view(x.size(0), -1)
        x2 = self.global_pool(feat_inc).view(x.size(0), -1)
        x3 = self.global_pool(feat_eff).view(x.size(0), -1)

        # 3. Concatenation (Triple Fusion)
        merged = torch.cat([x1, x2, x3], dim=1)

        # 4. Classification
        logits = self.classifier(merged)
        
        return {
            "logits": logits, 
            "feat": merged,
            "domain_logits": torch.zeros(x.size(0), 2).to(x.device),
            "seg": torch.zeros(x.size(0), 1, IMG_SIZE, IMG_SIZE).to(x.device)
        }

# --- 7. LOSS AND ADVERSARIAL HELPERS (From Notebook Cells 17-19) ---

class LabelSmoothingCE(nn.Module):
    def __init__(self, smoothing=0.1):
        super().__init__()
        self.s = smoothing
    def forward(self, logits, target):
        c = logits.size(-1)
        logp = F.log_softmax(logits, dim=-1)
        with torch.no_grad():
            true_dist = torch.zeros_like(logp)
            true_dist.fill_(self.s / (c - 1))
            true_dist.scatter_(1, target.unsqueeze(1), 1.0 - self.s)
        return (-true_dist * logp).sum(dim=-1).mean()

class FocalLoss(nn.Module):
    def __init__(self, gamma=1.5):
        super().__init__()
        self.gamma = gamma
    def forward(self, logits, target):
        prob = F.softmax(logits, dim=1)
        pt = prob.gather(1, target.unsqueeze(1)).squeeze(1)
        ce = F.cross_entropy(logits, target, reduction='none')
        loss = ((1 - pt) ** self.gamma) * ce
        return loss.mean()

def dice_loss(pred, target, smooth=1.0):
    pred = torch.sigmoid(pred)
    num = 2 * (pred * target).sum() + smooth
    den = pred.sum() + target.sum() + smooth
    return 1 - (num / den)

def seg_loss_fn(pred, mask):
    if pred.shape[-2:] != mask.shape[-2:]:
        pred = F.interpolate(pred, size=mask.shape[-2:], mode="bilinear", align_corners=False)
    return F.binary_cross_entropy_with_logits(pred, mask) + dice_loss(pred, mask)

class SupConLoss(nn.Module):
    def __init__(self, temperature=0.07):
        super().__init__()
        self.temperature = temperature
        self.cos = nn.CosineSimilarity(dim=-1)
    def forward(self, features, labels):
        device = features.device
        f = F.normalize(features, dim=1)
        sim = torch.matmul(f, f.T) / self.temperature
        labels = labels.contiguous().view(-1,1)
        mask = torch.eq(labels, labels.T).float().to(device)
        logits_max, _ = torch.max(sim, dim=1, keepdim=True)
        logits = sim - logits_max.detach()
        exp_logits = torch.exp(logits) * (1 - torch.eye(len(features), device=device))
        denom = exp_logits.sum(1, keepdim=True)
        pos_mask = mask - torch.eye(len(features), device=device)
        pos_exp = (exp_logits * pos_mask).sum(1)
        loss = -torch.log((pos_exp + 1e-8) / (denom + 1e-8) + 1e-12)
        valid = (pos_mask.sum(1) > 0).float()
        loss = (loss * valid).sum() / (valid.sum() + 1e-8)
        return loss

clf_loss_ce = LabelSmoothingCE(LABEL_SMOOTH)
clf_loss_focal = FocalLoss(gamma=1.5)
supcon_loss_fn = SupConLoss(temperature=0.07)

class GradReverse(Function):
    @staticmethod
    def forward(ctx, x, l):
        ctx.l = l
        return x.view_as(x)
    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.neg() * ctx.l, None

def grad_reverse(x, l=1.0):
    return GradReverse.apply(x, l)


# --- 8. DATA AUGMENTATION HELPERS (From Notebook Cell 20) ---

def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)

    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

def apply_mixup(x, y, alpha=MIXUP_ALPHA):
    lam = np.random.beta(alpha, alpha)
    idx = torch.randperm(x.size(0))
    mixed_x = lam * x + (1 - lam) * x[idx]
    y_a, y_b = y, y[idx]
    return mixed_x, y_a, y_b, lam

def apply_cutmix(x, y, alpha=CUTMIX_ALPHA):
    lam = np.random.beta(alpha, alpha)
    idx = torch.randperm(x.size(0))
    bbx1, bby1, bbx2, bby2 = rand_bbox(x.size(), lam)
    new_x = x.clone()
    new_x[:, :, bby1:bby2, bbx1:bbx2] = x[idx, :, bby1:bby2, bbx1:bbx2]
    lam_adjusted = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (x.size(-1) * x.size(-2)))
    return new_x, y, y[idx], lam_adjusted


# --- 9. DATA LOADING SETUP (From Notebook Cell 8) ---
info_dir = "/kaggle/input/cod10k/COD10K-v3/Info"
train_dir_cod = "/kaggle/input/cod10k/COD10K-v3/Train" 
test_dir_cod = "/kaggle/input/cod10k/COD10K-v3/Test"  
    
train_cam_txt = os.path.join(info_dir, "CAM_train.txt")
train_noncam_txt = os.path.join(info_dir, "NonCAM_train.txt")
test_cam_txt = os.path.join(info_dir, "CAM_test.txt")
test_noncam_txt = os.path.join(info_dir, "NonCAM_test.txt")

info_dir2 = "/kaggle/input/camo-coco/CAMO_COCO/Info"
train_cam_txt2 = os.path.join(info_dir2, "camo_train.txt")
train_noncam_txt2 = os.path.join(info_dir2, "non_camo_train.txt")
test_cam_txt2 = os.path.join(info_dir2, "camo_test.txt")
test_noncam_txt2 = os.path.join(info_dir2, "non_camo_test.txt")

train_dir_camo_cam = "/kaggle/input/camo-coco/CAMO_COCO/Camouflage"
train_dir_camo_noncam = "/kaggle/input/camo-coco/CAMO_COCO/Non_Camouflage"

testing_info_file = "/kaggle/input/testing-dataset/Info/image_labels.txt"
testing_images_dir = "/kaggle/input/testing-dataset/Images"

# --- MOCKING DATASET LOADING FOR RUNNABILITY ---
try:
    testing_image_paths, testing_labels = load_testing_dataset_info(testing_info_file, testing_images_dir)
    train_paths, val_paths, train_labels, val_labels = train_test_split(
        testing_image_paths, testing_labels, test_size=0.8, random_state=SEED
    )
    ALL_ROOT_DIRS = [
        train_dir_cod,       
        test_dir_cod,       
        train_dir_camo_cam,  
        train_dir_camo_noncam
    ]
    ALL_TRAIN_TXTS = [
        train_cam_txt2, train_noncam_txt2,
    ]
    ALL_VAL_TXTS = []
    
    train_ds = MultiDataset(
        root_dirs=ALL_ROOT_DIRS, 
        txt_files=ALL_TRAIN_TXTS,               
        testing_image_paths=train_paths,        
        testing_labels=train_labels,            
        weak_transform=weak_tf, 
        strong_transform=strong_tf, 
        use_masks=USE_SEGMENTATION
    )
    val_ds = MultiDataset(
        root_dirs=ALL_ROOT_DIRS,  
        txt_files=ALL_VAL_TXTS,                 
        testing_image_paths=val_paths,          
        testing_labels=val_labels,              
        weak_transform=val_tf, 
        strong_transform=None, 
        use_masks=USE_SEGMENTATION
    )

    def build_weighted_sampler(dataset):
        labels = [sample[1] for sample in dataset.samples]  
        counts = Counter(labels)
        total = len(labels)
        if len(counts) <= 1:
            weights = [1.0] * total
        else:
            class_weights = {c: total / (counts[c] * len(counts)) for c in counts}
            weights = [class_weights[lbl] for lbl in labels]
        return WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)

    train_sampler = build_weighted_sampler(train_ds)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=train_sampler, num_workers=NUM_WORKERS)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

except RuntimeError as e:
    print(f"⚠️ Warning: Cannot access Kaggle paths ({e}). Using Mock DataLoaders for demonstration.")
    
    class MockDataset(Dataset):
        def __init__(self, num_samples, num_classes=2, img_size=IMG_SIZE):
            self.num_samples = num_samples
            self.data = torch.randn(num_samples, 3, img_size, img_size)
            self.labels = torch.randint(0, num_classes, (num_samples,))
            self.masks = torch.randint(0, 2, (num_samples, 1, img_size, img_size)).float()

        def __len__(self):
            return self.num_samples

        def __getitem__(self, idx):
            return self.data[idx], self.data[idx].clone(), self.labels[idx], self.masks[idx]

    train_ds = MockDataset(num_samples=14150) 
    val_ds = MockDataset(num_samples=6606)   
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    
    print(f"Mock DataLoaders initialized with {len(train_ds)} train and {len(val_ds)} val samples.")


# --- 10. MODEL INSTANTIATION AND OPTIMIZER SETUP (From Notebook Cell 20) ---

model = TripleFusionModel(num_classes=2).to(device)

# Update parameter groups for Optimizer
backbone_names = ['densenet_base', 'inception_base', 'efficient_base']
backbone_params = []
head_params = []
for name, param in model.named_parameters():
    if any(k in name for k in backbone_names):
        backbone_params.append(param)
    else:
        head_params.append(param)

def get_cosine_with_warmup_scheduler(optimizer, warmup_epochs, total_epochs, last_epoch=-1):
    def lr_lambda(epoch):
        if epoch < warmup_epochs:
            return float(epoch) / float(max(1.0, warmup_epochs))
        t = (epoch - warmup_epochs) / float(max(1, total_epochs - warmup_epochs))
        return 0.5 * (1.0 + math.cos(math.pi * t))
    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda, last_epoch=last_epoch)
opt = torch.optim.AdamW([
    {'params': backbone_params, 'lr': LR / 10}, # Backbones usually need a smaller LR
    {'params': head_params, 'lr': LR}
], weight_decay=1e-4)
scheduler = get_cosine_with_warmup_scheduler(opt, WARMUP_EPOCHS, EPOCHS)
scaler = torch.cuda.amp.GradScaler(enabled=(device=="cuda"))

print(f"\nModel instantiated. LR: {LR}, Epochs: {EPOCHS}.")
print(f"Backbones are initially frozen for {FREEZE_EPOCHS} epochs.")

# --- 11. TRAINING LOOP (Adapted from Notebook Cell 21) ---

def compute_combined_clf_loss(logits, targets, mix_info=None, use_focal=False):
    if mix_info is None:
        if use_focal:
            return clf_loss_focal(logits, targets)
        else:
            return clf_loss_ce(logits, targets)
    else:
        y_a, y_b, lam = mix_info
        if use_focal:
            loss = lam * F.cross_entropy(logits, y_a) + (1 - lam) * F.cross_entropy(logits, y_b)
        else:
            loss = lam * clf_loss_ce(logits, y_a) + (1 - lam) * clf_loss_ce(logits, y_b)
        return loss

best_vf1 = 0.0
best_epoch = 0
patience_count = 0

for epoch in range(1, EPOCHS+1):
    # --- Freeze/Unfreeze Logic ---
    if epoch <= FREEZE_EPOCHS:
        for name, p in model.named_parameters():
            if any(k in name for k in ['densenet_base', 'mobilenet_base']):
                p.requires_grad = False
    if epoch == FREEZE_EPOCHS + 1:
        print(f"--- Unfreezing all 3 backbones at epoch {epoch} ---")
        for name, p in model.named_parameters():
            if any(k in name for k in backbone_names):
                p.requires_grad = True

    model.train()
    running_loss = 0.0
    y_true, y_pred = [], []
    n_batches = 0
    opt.zero_grad() 
    
    for i, (weak_imgs, strong_imgs, labels, masks) in enumerate(tqdm(train_loader, desc=f"Train {epoch}/{EPOCHS}")):
        weak_imgs = weak_imgs.to(device); strong_imgs = strong_imgs.to(device)
        labels = labels.to(device)
        if masks is not None:
            masks = masks.to(device)

        imgs = weak_imgs

        mix_info = None
        rand = random.random()
        if rand < PROB_MIXUP:
            imgs, y_a, y_b, lam = apply_mixup(imgs, labels)
            mix_info = (y_a.to(device), y_b.to(device), lam)
        elif rand < PROB_MIXUP + PROB_CUTMIX:
            imgs, y_a, y_b, lam = apply_cutmix(imgs, labels)
            mix_info = (y_a.to(device), y_b.to(device), lam)

        with torch.autocast(device_type=device, enabled=(device=="cuda")):
            out = model(imgs) 
            logits = out["logits"]
            
            clf_loss = compute_combined_clf_loss(logits, labels, mix_info=mix_info, use_focal=False)

            # Auxiliary losses are set to 0 as the user's simple fusion model excludes these complex heads.
            seg_loss = 0.0
            supcon_loss = 0.0 
            cons_loss = 0.0   
            dom_loss = 0.0

            total_loss = clf_loss + GAMMA_SEG * seg_loss + BETA_SUPCON * supcon_loss + ETA_CONS * cons_loss + ALPHA_DOM * dom_loss
            
            total_loss = total_loss / ACCUMULATION_STEPS 

        scaler.scale(total_loss).backward()

        if (i + 1) % ACCUMULATION_STEPS == 0:
            scaler.unscale_(opt)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(opt)
            scaler.update()
            opt.zero_grad() 

        running_loss += total_loss.item() * ACCUMULATION_STEPS
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(logits.argmax(1).cpu().numpy())
        n_batches += 1

    if n_batches % ACCUMULATION_STEPS != 0:
        scaler.unscale_(opt)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(opt)
        scaler.update()
        opt.zero_grad()

    scheduler.step()

    acc = accuracy_score(y_true, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="macro", zero_division=0)
    print(f"[Epoch {epoch}] Train Loss: {running_loss/max(1,n_batches):.4f} Acc: {acc:.4f} Prec: {prec:.4f} Rec: {rec:.4f} F1: {f1:.4f}")

    # -------------------
    # VALIDATION
    # -------------------
    model.eval()
    val_y_true, val_y_pred = [], []
    val_loss = 0.0
    with torch.no_grad():
        for weak_imgs, _, labels, masks in val_loader:
            imgs = weak_imgs.to(device)
            labels = labels.to(device)
            if masks is not None:
                masks = masks.to(device)

            out = model(imgs)
            logits = out["logits"]
            
            loss = compute_combined_clf_loss(logits, labels, mix_info=None, use_focal=False)
            if USE_SEGMENTATION and (masks is not None):
                loss += GAMMA_SEG * seg_loss_fn(out["seg"], masks) 
            val_loss += loss.item()

            val_y_true.extend(labels.cpu().numpy())
            val_y_pred.extend(logits.argmax(1).cpu().numpy())

    vacc = accuracy_score(val_y_true, val_y_pred)
    vprec, vrec, vf1, _ = precision_recall_fscore_support(val_y_true, val_y_pred, average="macro", zero_division=0)
    print(f"[Epoch {epoch}] Val Loss: {val_loss/max(1,len(val_loader)):.4f} Acc: {vacc:.4f} Prec: {vprec:.4f} Rec: {vrec:.4f} F1: {vf1:.4f}")

    if vf1 > best_vf1:
        best_vf1 = vf1
        best_epoch = epoch
        torch.save({
            "epoch": epoch,
            "model_state": model.state_dict(),
            "opt_state": opt.state_dict(),
            "best_vf1": best_vf1
        }, SAVE_PATH)
        patience_count = 0
        print(f"Saved best model at epoch {epoch} (F1 {best_vf1:.4f})")
    else:
        patience_count += 1
        if patience_count >= EARLY_STOPPING_PATIENCE:
            print("Early stopping triggered.")
            break

print("\nTraining finished. Best val F1:", best_vf1, "at epoch", best_epoch)

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Device: cuda
✅ Loaded 3651 samples from 4 root directories.
✅ Loaded 6606 samples from 4 root directories.
Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /root/.cache/torch/hub/checkpoints/densenet201-c1103571.pth


100%|██████████| 77.4M/77.4M [00:00<00:00, 167MB/s]


Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


100%|██████████| 104M/104M [00:00<00:00, 208MB/s] 


...

[Epoch 14] Train Loss: 0.4304 Acc: 0.8332 Prec: 0.8333 Rec: 0.8331 F1: 0.8331


[Epoch 14] Val Loss: 1.1918 Acc: 0.9986 Prec: 0.9986 Rec: 0.9986 F1: 0.9986
Saved best model at epoch 14 (F1 0.9986)


Train 15/15: 100%|██████████| 457/457 [02:02<00:00,  3.73it/s]

[Epoch 15] Train Loss: 0.4314 Acc: 0.8381 Prec: 0.8384 Rec: 0.8383 F1: 0.8381


[Epoch 15] Val Loss: 1.1908 Acc: 0.9986 Prec: 0.9986 Rec: 0.9986 F1: 0.9986

Training finished. Best val F1: 0.9986375856644882 at epoch 14
